# METBK

### Purpose
The purpose of this notebook is to calculate the gross range user min/max values for populating QARTOD parameter tables for data streams for OOI - CGSN data streams for the ASIMET bulk-meteorology measurements (METBK). The METBK package measures a collection of atmospheric parameters: wind speed (u-, v-vectors), air temperature, barometric pressure, short- and longwave irradiance, relative humidity, and precipitation. It also measures the sea surface salinity and temperature. There are number of base-and-derived data products delivered for the METBK: we test only the base measurements, and ignore the hourly-averaged as well as derived flux products. The METBK is deployed only on surface moorings.
  
### Test Parameters

| Variable | Sensor | Range | Resolution | Accuracy |
|----------|--------|-------|------------|----------|
| met_windavg_mag_corr_east | Gill Instruments WindObserver II Anemometer | 0 - 65 m/s, 0 - 360 deg | 0.01 m/s | 2%, 2 deg |
| met_windavg_mag_corr_north| Gill Instruments WindObserver II Anemometer | 0 - 65 m/s, 0 - 360 deg | 0.01 m/s | 2%, 2 deg |
| air_temperature | Rotronic MP-101A | -40C - 60C | 0.02C | 0.05C |
| relative_humidity | Rotronic MP-101A | 0 - 100%RH | 0.01%RH | 1%RH |
| barometric_pressure | Heise DXD (Dresser Instruments) | 850 - 1050 mb | 0.01 mb | 0.2 mb |
| longwave_irradiance | Eppley Precision INfrared Radiometer (PIR) | 0 - 700 W/m2 | 0.1 W/m2 | 2 W/m2 |
| shortwave_irradiance| Eppley Precision Spectral Pyranometer (PSP)| 0 - 2800 W/m2 | 0.1 W/m2 | 2 W/m2 |
| precipitation | RM Young 50202 Self-siphoning rain gauge | 0 - 500 mm | 0.1 mm | 1 mm/hr |
| sea_surface_conductivity | SBE 37 | 0 - 7 S/m | 0.00001 S/m | 0.0003 S/m |
| met_salsurf | SBE 37 | 0 - 42 psu | - | - |
| sea_surface_temperature | SBE 37 | -5 - 35C | 0.0001C | 0.002C (-5 - 35C) |

**Special Case: Wind Data**

For Wind Data, from the QARTOD Wind Manual: 
```
Wind speed (WS) is used in the descriptions and examples, but the tests apply equally to direction
and gust in most cases. A discontinuity in wind direction is caused when the wind veers through north,
stepping from 359° to 0° and complicating the application of some of these tests. Operators may choose to
conduct wind direction tests on the u and v wind direction components to circumvent the problem.
```
* ```eastward_wind_velocity```: windspeed in the eastward direction, for the METBK instrument, not corrected for magnetic declination
* ```northward_wind_velocity```: windspeed in the northward direction, for the METBK instrument, not corrected for magnetic declination

For the Gill Instruments WindObserver II 2-axis sonic anemometer operates on the following prinicple of operation: 
```
The WindObserver II measures the times taken for an ultrasonic pulse of sound to travel
from the North transducer to the South transducer, and compares it with the time for a
pulse to travel from S to N transducer. Likewise times are compared between West and
East, and E and W transducer.
If, for example, a North wind is blowing, then the time taken for the pulse to travel from N
to S will be faster than from S to N, whereas the W to E, and E to W times will be the
same. The wind speed and direction (and the speed of sound) can then be calculated from
the differences in the times of flight on each axis. This calculation is independent of
factors such as temperature.
```
This suggests that the eastward and northward wind velocities should be the tested parameters. Additionally, the instrument used to measure wind speed (the Gill WindObserver II) has a range of 0-65 m/s, which is taken as the absolute maximum. Consequently, that means when the wind is blowing at a 45$^{\circ}$ angle to the east and north instrument sensors that the maximum individual component speeds are $max(u, v) = \sqrt{(ws^{2})/2}$

#### Import libraries

In [ ]:
import os, sys, datetime, pytz, re
import dateutil.parser as parser
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import gc
import json
warnings.filterwarnings("ignore")

In [ ]:
from dask.diagnostics import ProgressBar

#### Import the ```ooinet``` M2M toolbox

In [ ]:
# Import the OOINet M2M tool
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/")
from ooinet import M2M
from ooinet.utils import convert_time, ntp_seconds_to_datetime, unix_epoch_time
from ooinet.Instrument.common import process_file, add_annotation_qc_flag

#### Import ```ooi_data_explorations``` toolbox

In [ ]:
sys.path.append("/home/areed/Documents/OOI/oceanobservatories/ooi-data-explorations/python/")
from ooi_data_explorations.common import get_annotations, get_vocabulary, load_gc_thredds
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.uncabled.process_metbk import metbk_datalogger
from ooi_data_explorations.qartod.qc_processing import identify_blocks, create_annotations, process_gross_range, \
    process_climatology, parse_qc, inputs, ANNO_HEADER, CLM_HEADER, GR_HEADER

#### Import plotting and visualization tools

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sys.path.append("../../qartod/")
import qartod.plotting

---
## Identify Data Streams
This section is necessary to identify all of the data stream associated with a specific instrument. This can be done by querying UFrame and iteratively walking through all of the API endpoints. The results are saved into a csv file so this step doesn't have to be repeated each time.

First, set the instrument to search for using OOI terminology:

In [ ]:
instrument = "METBK"

### Query OOINet for Data Streams <br>
First check if the datasets have already been downloaded; if not, use the ```M2M.search_datasets``` tool to search the OOINet API and return a table of all of the available datasets for the given instruments.

In [1]:
try:
    datasets = pd.read_csv("../data/METBK_datasets.csv")
except:
    datasets = M2M.search_datasets(instrument="METBK", English_names=True)
    # Save the datasets
    datasets.to_csv("../data/METBK_datasets.csv", index=False)

NameError: name 'M2M' is not defined

Separate out the CGSN datasets from the EA datasets:

In [ ]:
cgsn = datasets["array"].apply(lambda x: True if x.startswith(("CP","GA","GI","GP","GS")) else False)
datasets = datasets[cgsn]
datasets

---
## Single Reference Designator
The reference designator acts as a key for an instrument located at a specific location. First, select a reference designator (refdes) to request data from OOINet.

In [ ]:
reference_designators = sorted(datasets["refdes"])
print("Number of reference designators: " + str(len(reference_designators)))
for refdes in reference_designators:
    print(refdes)

In [ ]:
k=0
refdes = reference_designators[k]
print(refdes)

#### Sensor Vocab
The vocab provides information about the instrument model and type, its location (with descriptive names), depth, and manufacturer. Get the vocab for the given reference designator.

In [ ]:
vocab = M2M.get_vocab(refdes)
vocab

#### Sensor Deployments
Download the deployment information for the selected reference designator:

In [ ]:
deployments = M2M.get_deployments(refdes)
deployments

#### Sensor Data Streams
Next, select the specific data streams for the given reference designator

In [ ]:
datastreams = M2M.get_datastreams(refdes)
datastreams

---
## Metadata 
The metadata contains the following important key pieces of data for each reference designator: **method**, **stream**, **particleKey**, and **count**. The method and stream are necessary for identifying and loading the relevant dataset. The particleKey tells us which data variables in the dataset we should be calculating the QARTOD parameters for. The count lets us know which dataset (the recovered instrument, recovered host, or telemetered) contains the most data and likely has the best record to use to calculate the QARTOD tables. 

In [ ]:
metadata = M2M.get_metadata(refdes)
metadata

#### Sensor Parameters
Each instrument returns multiple parameters containing a variety of low-level instrument output and metadata. However, we are interested in science-relevant parameters for calculating the relevant QARTOD test limits. We can identify the science parameters based on the preload database, which designates the science parameters with a "data level" of L1 or L2. 

Consequently, we through several steps to identify the relevant parameters. First, we query the preload database with the relevant metadata for a reference designator. Then, we filter the metadata for the science-relevant data streams. 

In [ ]:
def filter_science_parameters(metadata):
    """This function returns the science parameters for each datastream"""
    
    def filter_parameter_ids(pdId, pid_dict):
        data_level = pid_dict.get(pdId)
        if data_level is not None:
            if data_level > 0:
                return True
            else:
                return False
        else:
            return False
    
    # Filter the parameters for processed science parameters
    data_levels = M2M.get_parameter_data_levels(metadata)
    mask = metadata["pdId"].apply(lambda x: filter_parameter_ids(x, data_levels))
    metadata = metadata[mask]

    return metadata

def filter_metadata(metadata):
    science_vars = filter_science_parameters(metadata)
    # Next, eliminate the optode temperature from the stream
    mask = science_vars["particleKey"].apply(lambda x: False if "temp" in x else True)
    science_vars = science_vars[mask]
    science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
    science_vars = science_vars.reset_index()
    science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
    science_vars = science_vars.explode(column="particleKey")
    return science_vars

In [ ]:
science_vars = filter_science_parameters(metadata)
science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
science_vars = science_vars.reset_index()
science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
science_vars

---
## Load Data
When calculating the QARTOD data tables, we only want to utilize the most complete data record available for a given reference designator. We do this by getting all the available data streams, loading the data, and then combining them into a single dataset. 

In [ ]:
def trim_overlaps(ds, deployments):
    """Trim overlapping deployment data (necessary to use xr.open_mfdataset)"""
    # --------------------------------
    # Second, get the deployment times
    deployments = deployments.sort_values(by="deploymentNumber")
    deployments = deployments.set_index(keys="deploymentNumber")
    # Shift the start times by (-1) 
    deployEnd = deployments["deployStart"].shift(-1)
    # Find where the deployEnd times are earlier than the deployStart times
    mask = deployments["deployEnd"] > deployEnd
    # Wherever the deployEnd times occur after the shifted deployStart times, replace those deployEnd times
    deployments["deployEnd"][mask] = deployEnd[mask]
    deployments["deployEnd"] = deployments["deployEnd"].apply(lambda x: pd.to_datetime(x))
    
    # ---------------------------------
    # With the deployments info, can write a preprocess function to filter 
    # the data based on the deployment number
    depNum = np.unique(ds["deployment"])
    deployInfo = deployments.loc[depNum]
    deployStart = deployInfo["deployStart"].values[0]
    deployEnd = deployInfo["deployEnd"].values[0]
    
    # Select the dataset data which falls within the specified time range
    ds = ds.sel(time=slice(deployStart, deployEnd))
    
    return ds

def preprocess(ds):
    ds = process_file(ds)
    ds = metbk_datalogger(ds)
    ds = trim_overlaps(ds, deployments)
    gc.collect()
    return ds

**Filter out the "hourly" datastreams; use only the regular dataset**

In [ ]:
mask = datastreams["stream"].apply(lambda x: False if "hourly" in x else True)
datastreams[mask]

---
## Download Data
To access data, there are two applicable methods. The first is to download the data and save the netCDF files locally. The second is to access and process the files remotely on the THREDDS server, without having to download the data.

In [ ]:
# Get the available datasets
for index in datastreams[mask].index:
    # Get the method and stream
    method = datastreams.loc[index]["method"]
    stream = datastreams.loc[index]["stream"]

    # Get the URL - first try the goldCopy thredds server
    thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

    # Get the catalog
    catalog = M2M.get_thredds_catalog(thredds_url)

    # Clean the catalog
    catalog = M2M.clean_catalog(catalog, stream, deployments)
    
    # Get the links to the THREDDs server and load the data
    dodsC = M2M.URLS["goldCopy_dodsC"]
    if method == "telemetered":
        tele_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess, parallel=True)
    elif method == "recovered_host":
        host_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            host_data = xr.open_mfdataset(host_files, preprocess=preprocess, parallel=True)
    else:
        pass

**Combine the datasets into a single dataset**

In [ ]:
data = combine_datasets(tele_data, host_data, None, None)
data

**Clean up workspace variables and free up memory**

In [ ]:
host_data.close()
tele_data.close()
del tele_data, host_data
gc.collect()

---
## Process the DOSTA
The METBK data needs reprocessing before we can generate the qartod tables. This includes removing data identified by annotations as bad and filtering out data that, based on HITL review, is outside of the expected instrument calibration ranges. Once these steps are done, then the cleaned up datasets can be used to generate more-accurate QARTOD tables.

#### Process existing qc tests

In [ ]:
data = parse_qc(data)
data

In [ ]:
for var in data.variables:
    if "qc_summary" in var:
        if "shortwave_irradiance" in var:
            continue
        # Get variable name
        var_name = var.split("_qc")[0]
        # Now nan-out the appropriate 
        mask = (data[var] >= 3)
        data[var_name][mask] = np.nan

#### Annotations
The annotations associated with a specific reference designator may contain relevant information on the performance or reliability of the data for a given dataset. The annotations are downloaded from OOINet as a json and processed into a pandas dataframe. Each annotation may apply to the entire dataset, to a specific stream, or to a specific variable. With the downloaed annotations, we can use the information contained in the ```qcFlag``` column to translate the annotations into QC flags, which can then be used to filter out bad data. 

In [ ]:
annotations = M2M.get_annotations(refdes)
annotations

Pass in the annotations and the dataset to add the annotation ```qcFlag``` values to the dataset

In [ ]:
data = add_annotation_qc_flag(data, annotations)
data

Use the added ```rollup_annotations_qc_results``` values to filter out bad or suspect (```rollup_annotations_qc_results``` value of 3, 4, or 9) data from the dataset

In [ ]:
data = data.where(data.rollup_annotations_qc_results <= 3, drop=True)
data

#### Limit the data to data collected before 2021-01-01

In [ ]:
_, index = np.unique(data['time'], return_index=True)
data = data.isel(time=index)
data = data.sel(time=slice('2014-01-01T00:00:00', "2021-01-01T00:00:00"))
data

#### Specific filters for the METBK data

In [ ]:
mask = data["sea_surface_conductivity"] < 3.0
data["sea_surface_conductivity"][mask] = np.nan
data["sea_surface_salinity"][mask] = np.nan

mask = (data["sea_surface_temperature"] < 0) | (data["sea_surface_temperature"] > 20)
data["sea_surface_temperature"][mask] = np.nan

#mask = (data["air_temperature"] > 20) 
#data["sea_surface_temperature"][mask] = np.nan

mask = (data["barometric_pressure"] < 850) | (data["barometric_pressure"] > 1050)
data["barometric_pressure"][mask] = np.nan

mask = (data["relative_humidity"] > 105) | (data["relative_humidity"] < 0)
data["relative_humidity"][mask] = np.nan

mask = (data["shortwave_irradiance"] > 2800) | (data["shortwave_irradiance"] < 0)
data["shortwave_irradiance"][mask] = np.nan

mask = data["sea_surface_salinity"] < 33.6
data["sea_surface_salinity"][mask] = np.nan

**Plot a timeseries for each test parameters**

In [ ]:
subset = sorted(np.random.choice(data.time, 100000, replace=False))
for param in list(test_parameters.keys()):
    if param in data.variables:
        sensor_range = test_parameters.get(param)
        fig, ax = plot_variable(data.sel(time=subset), param)
        #ax.set_ylim(sensor_range)

---
## Gross Range
The Gross Range QARTOD test consists of two parameters: a fail range which indicates when the data is bad, and a suspect range which indicates when data is either questionable or interesting. The fail range values are set based upon the instrument/measurement and associated calibration. For example, the conductivity sensors are calibration for measurements between 0 (freshwater) and 9 (highly-saline waters). The suspect range values are calculated based on the mean of the available data $\pm$3$\sigma$.

In [ ]:
from ooi_data_explorations.qartod.gross_range import GrossRange
from ooi_data_explorations.qartod.plotting import *
from ooi_data_explorations.qartod.qc_processing import format_gross_range, format_climatology

#### Test Parameters & Sensor Ranges
Map out the data variables in the data set to the data stream inputs and the associated sensor ranges

In [ ]:
test_parameters = {
    "eastward_wind_velocity": [-45, 45],
    "northward_wind_velocity": [-45, 45],
    "air_temperature": [ -40, 60],
    "relative_humidity": [0, 100],
    "barometric_pressure": [850, 1050],
    "longwave_irradiance": [0, 700],
    "shortwave_irradiance": [0, 2800],
    "precipitation": [0, 500],
    "sea_surface_conductivity": [0, 7],
    "sea_surface_temperature": [-5, 35],
    "sea_surface_salinity": [0, 42]
}

**Calculate the Gross Range Values**

In [ ]:
site, node, sensor = refdes.split("-", 2)
gross_range_table = pd.DataFrame()

for param in test_parameters:
    sensor_range = test_parameters.get(param)
    if param == "sea_surface_salinity":
        inp = "met_salsurf"
    else:
        inp = param
    
    if param in data.variables:
        print(f"##### Calculating gross range for {param} #####")
        # Check if there is enough data
        if len(data[param].dropna(dim="time")) < 100:
            user_range = [np.nan, np.nan]
            source = "Not enough data to calculate user range."
        else:
            # If barometric pressure, have to remove the data where 900 appears since that is bad
            if param == "barometric_pressure":
                mask = (data[param] == 900)
                data[param][mask] = np.nan
            # If sea_surface_conductivity, fail mode is -0.0003
            if param == "sea_surface_conductivity":
                mask = (data[param] == -0.0003)
                data[param][mask] = np.nan
            gross_range = GrossRange(sensor_range[0], sensor_range[1])
            gross_range.fit(data, param, check_normality=True)
            user_range = [gross_range.suspect_min, gross_range.suspect_max]
            source = gross_range.source
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            if stream == "metbk_hourly":
                pass
            else:
                qc_dict = format_gross_range(inp, sensor_range, user_range, site, node, sensor, stream, source)
                gross_range_table = gross_range_table.append(qc_dict, ignore_index=True)
            
        # Plot the result
        try:
            fig, ax = plot_gross_range(data, param, gross_range)
            ymin, ymax = ax.get_ylim()
            if ymin < sensor_range[0]:
                ymin = sensor_range[0]
            if ymax > sensor_range[1]:
                ymax = sensor_range[1]
            ax.set_ylim((ymin, ymax))
            #ax.set_ylim((200, 600))
        except:
            pass

**Add the stream name and the source comments**

In [ ]:
gross_range_table['notes'] = ('User range based on data collected through {}.'.format("2021-01-01"))
gross_range_table

**Check the results**

In [ ]:
for ind in gross_range_table.index:
    print(gross_range_table.loc[ind]["qcConfig"])

**Save the gross range table**

In [ ]:
# Save the results
gross_range_table.to_csv(f"../results/gross_range/{refdes}.csv", index=False, columns=GR_HEADER)

---
## Climatology
For the climatology QARTOD test, First, we bin the data by month and take the mean. The binned-montly means are then fit with a 2 cycle harmonic via Ordinary-Least-Squares (OLS) regression. Ranges are calculated based on the 3$\sigma$ calculated from the OLS-fitting.  

In [ ]:
from ooi_data_explorations.qartod.climatology import Climatology

In [ ]:
def make_climatology_table(ds, param, tinp, zinp, sensor_range, depth_bins):
    """Function which calculates the climatology table based on the """
    
    climatologyTable = pd.DataFrame()
    
    if depth_bins is None:
        # Filter out the data outside the sensor range
        m = (ds[param] > sensor_range[0]) & (ds[param] < sensor_range[1]) & (~np.isnan(ds[param]))
        param_data = ds[param][m]
        
        # Fit the climatology for the selected data
        pmin, pmax = [0, 0]
        
        try:
            climatology = Climatology()
            climatology.fit(param_data)

            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Calculate the climatology data
            vmin = climatology.monthly_fit - climatology.monthly_std*3
            vmin = np.floor(vmin*10000)/10000
            for vind in vmin.index:
                if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                    vmin[vind] = sensor_range[0]
            vmax = climatology.monthly_fit + climatology.monthly_std*3
            for vind in vmax.index:
                if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                    vmax[vind] = sensor_range[1]
            vmax = np.floor(vmax*10000)/10000
            vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
            vspan = vdata.values.reshape(1,-1)

            # Build the climatology dataframe
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

        except:
            # Here is where to create nans if insufficient data to fit
            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Create a series filled with nans
            vals = []
            for i in np.arange(len(tspan)):
                vals.append([np.nan, np.nan])
            vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

            # Add to the data
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))
            
        del ds, vspan, tspan, zspan
        gc.collect()
        
    else:        
    # Iterate through the depth bins to calculate the climatology for each depth bin
        for dbin in depth_bins:
            # Get the pressure range to bin from
            pmin, pmax = dbin[0], dbin[1]

            # Select the data from the pressure range
            bin_data = data.where((data[zinp] >= pmin) & (data[zinp] <= pmax), drop=True)

            # sort based on time and make sure we have a monotonic dataset
            bin_data = bin_data.sortby('time')
            _, index = np.unique(bin_data['time'], return_index=True)
            bin_data = bin_data.isel(time=index)

            # Filter out the data outside the sensor range
            m = (bin_data[param] > sensor_range[0]) & (bin_data[param] < sensor_range[1]) & (~np.isnan(bin_data[param]))
            param_data = bin_data[param][m]

            # Fit the climatology for the selected data
            try:
                climatology = Climatology()
                climatology.fit(param_data)

                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Calculate the climatology data
                vmin = climatology.monthly_fit - climatology.monthly_std*3
                vmin = np.floor(vmin*10000)/10000
                for vind in vmin.index:
                    if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                        vmin[vind] = sensor_range[0]
                vmax = climatology.monthly_fit + climatology.monthly_std*3
                vmax = np.floor(vmax*10000)/10000
                for vind in vmax.index:
                    if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                        vmax[vind] = sensor_range[1]
                vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
                vspan = vdata.values.reshape(1,-1)

                # Build the climatology dataframe
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            except:
                # Here is where to create nans if insufficient data to fit
                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Create a series filled with nans
                vals = []
                for i in np.arange(len(tspan)):
                    vals.append([np.nan, np.nan])
                vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

                # Add to the data
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            del climatology, bin_data, vspan, tspan, zspan
            gc.collect()
    
    return climatologyTable, climatology

**Get the depth bins and filter based on max depth.** <br>
For the ```METBK``` which are only deployed on Surface Moorings, there is no depth bins needed.

In [ ]:
depth_bins = None

In [ ]:
# Initialize the climatology lookup table
climatologyLookup = pd.DataFrame()

# Setup the Table Header
TBL_HEADER = ["[1,1]","[2,2]","[3,3]","[4,4]","[5,5]","[6,6]","[7,7]","[8,8]","[9,9]","[10,10]","[11,11]","[12,12]"]

# Set the subsite-node-sensor
subsite, node, sensor = refdes.split("-", 2)

# Iterate through the parameters
for param in test_parameters:
    if param in data.variables:
        if param == "sea_surface_salinity":
            inp = "met_salsurf"
        else:
            inp = param
        # ----------------- Depth tables ---------------------
        # Get the sensor range of the parameter to test
        print(f"##### Calculating climatology for {param} #####")
        sensor_range = test_parameters.get(param)
        
        # Generate the climatology table with the depth bins
        climatologyTable, climatology = make_climatology_table(data, param, "time", "depth", sensor_range, depth_bins)
        
        # Get the variance
        try:
            variance = float(np.round(climatology.regression['variance_explained']*100, 1))
        except:
            variance = 0.0

        # Create the tableName
        tableName = f"{refdes}-{param}.csv"
        
        # Save the results
        climatologyTable.to_csv(f"../results/climatology/climatology_tables/{tableName}", header=TBL_HEADER)
        
        # ------------------ Lookup tables ------------------
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            if stream == "metbk_hourly":
                pass
            else:
                qc_dict = {
                    "subsite": subsite,
                    "node": node,
                    "sensor": sensor,
                    "stream": stream,
                    "parameters": {
                        "inp": inp,
                        "tinp": "time",
                        "zinp": "depth",
                    },
                    "climatologyTable": f"climatology_tables/{refdes}-{param}.csv",
                    "source": f"The variance explained by the climatology model is {variance}%.",
                    "notes": "Climatology based on available data through 2021-01-01."
                }
                # Append to the lookup table
                climatologyLookup = climatologyLookup.append(qc_dict, ignore_index=True)
            
        # ------------------ Plot the climatology ------------------
        if data[param].time.size > 100000:
            try:
                subset = sorted(np.random.choice(data.time, 100000, replace=False))
                subset_data = data.sel(time=subset)
                fig, ax = plot_climatology(subset_data, param, climatology)
                ax.set_ylim((sensor_range[0], sensor_range[1]))
                del subset, subset_data
                gc.collect()
            except:
                pass
        else:
            try:
                fig, ax = plot_climatology(data, param, climatology)
                ax.set_ylim((sensor_range[0], sensor_range[1]))
            except:
                pass

**Check the last climatologyTable for reasonableness**

In [ ]:
climatologyTable

**Check the climatologyLookup table that all the entries made it in**

In [ ]:
climatologyLookup

**Save the climatologyLookup table**

In [ ]:
# Save the lookup table results
climatologyLookup.to_csv(f"../results/climatology/{refdes}.csv", index=False, columns=CLM_HEADER)